# Dask with TensorFlow Assignment

The following assignment is based on [Keras and Tensorflow Dask Example](https://ml.dask.org/keras.html).

## Installation

In [ ]:
# Dask requires msgpack<1.0 so we will install that first
!pip install msgpack==0.6.2

In [ ]:
# We would also need to install dask-ml to run ML algorithms from Dask
!pip install dask-ml

In [ ]:
!pip install dask distributed --upgrade

In [6]:
# The package SciKeras brings a Scikit-learn API to Keras. This allows Dask-ML to be used seamlessly with Keras models.
!pip install scikeras>=0.1.8

After running all three, make sure you restart the kernel.

## Usage

In [47]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential

def build_model(beta_1=0.9):
    # build a model with:
    # Flatten layer with size (28, 28)
    # Dense layer with 128 and relu activation 
    # Dense layer with 10 and softmax activation
    layers = [Flatten(input_shape=(28, 28)),
              Dense(128, activation='relu'),
              Dense(10, activation="softmax")]
    model = Sequential(layers)

    # Use Adam Optimizer and pass in the beta_1 variable
    # If you would like to add more parameters, make sure you modify the function parameters
    opt = tf.keras.optimizers.Adam(beta_1=beta_1)
    # Use SparseCategoricalCrossentropy as your loss function with from_logits as True 
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    # Compile the model with accuracy metric
    model.compile(loss=loss, optimizer=opt, metrics=["accuracy"])
    return model

Now, we can use the SciKeras to create a Scikit-learn compatible model:

In [48]:
from scikeras.wrappers import KerasClassifier
niceties = dict(verbose=False)
model = KerasClassifier(build_fn=build_model, beta_1=0.9, **niceties)

This model will work with all of Dask-ML: it can use NumPy arrays as inputs and obeys the Scikit-learn API. For example, it’s possible to use Dask-ML to do the following:
- Use Keras with Dask-ML’s model selection, including `HyperbandSearchCV`.
- Use Keras with Dask-ML’s `Incremental`.

If we want to tune `learning_rate`, `beta_1` and `beta_2`, SciKeras requires that we pass `learning_rate`, `beta_1` and `beta_2` at initialization:

SciKeras supports more model creation methods, including some that are backwards-compatible with Tensorflow. Refer to their documentation for details.

## Hyperparameter Optimization

If we wanted to, we could use the model above with `HyperbandSearchCV`. Let’s tune this model on the MNIST dataset:

In [49]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import numpy as np
from typing import Tuple

def get_mnist() -> Tuple[np.ndarray, np.ndarray]:
    (X_train, y_train), _ = mnist.load_data()
    X_train = X_train.astype("float32")
    X_train /= 255
    return X_train, y_train

And let’s perform the basic task of tuning our Adam implementation:

In [50]:
params = {"beta_1": [0.7, 0.8, 0.9]}
X, y = get_mnist()

Now, the search can be run:

In [51]:
from dask.distributed import Client
client = Client()

from dask_ml.model_selection import HyperbandSearchCV

# Run HyperbandSearchCV with model, params and max_iter as 3
# If you run a larger iteration or more parameters, you may run into issues depending on 
# how much compute power you have access to.
search = HyperbandSearchCV(model, params, max_iter=3)
search.fit(X, y)

/usr/local/lib/python3.6/dist-packages/distributed/node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 33183 instead
  http_address["port"], self.http_server.port


HyperbandSearchCV(estimator=KerasClassifier(beta_1=0.9, build_fn=<function build_model at 0x7f47c93a60d0>, verbose=False),
                  max_iter=3, parameters={'beta_1': [0.7, 0.8, 0.9]})